## Pipeline Parameters

In [0]:
# Audit Parameters
dbutils.widgets.text("job_id", "")
dbutils.widgets.text("job_name", "")
dbutils.widgets.text("job_start_date", "")
dbutils.widgets.text("job_start_datetime", "")
dbutils.widgets.text("task_run_id", "")
dbutils.widgets.text("task_name", "")

In [0]:
%run ../utils/loggers

In [0]:
# Standard library imports
import os

# Third-party library imports
from dotenv import load_dotenv
from pyspark.sql.functions import concat

In [0]:
load_dotenv()

catalog_name = os.getenv('DATABRICKS_CATALOG_NAME')
schema_name = os.getenv('DATABRICKS_SCHEMA_NAME')

source_table_name = ['kdayno_silver_reddit_top_posts_sentiment'
                     , 'kdayno_silver_reddit_hot_posts_sentiment'
                     , 'kdayno_bronze_sp500_companies']
target_table_name = 'kdayno_silver_reddit_all_posts'

# Audit Variables
job_id = dbutils.widgets.get('job_id')
job_name = dbutils.widgets.get('job_name')
job_start_date = dbutils.widgets.get('job_start_date')
job_start_datetime = dbutils.widgets.get('job_start_datetime')
task_run_id = dbutils.widgets.get('task_run_id')
task_name = dbutils.widgets.get('task_name')

## Pipeline Audit

In [0]:
audit_logger(job_id, job_name, 'None', job_start_date, job_start_datetime, task_run_id,  task_name, source_table_name, target_table_name)

etl_logger = etl_logger()

In [0]:
silver_reddit_all_posts_df = spark.sql(
    f""" 
    WITH all_posts_with_sentiment AS (
        SELECT *, 'top' AS post_category FROM {catalog_name}.{schema_name}.{source_table_name[0]}
        UNION ALL
        SELECT *, 'hot' AS post_category FROM {catalog_name}.{schema_name}.{source_table_name[1]}
    ),

    all_posts_with_sentiment_transformed AS (
        SELECT 
            company_name
            , post_id
            , post_category
            , CONCAT('r/', subreddit) AS subreddit
            , created_utc AS created_date_utc
            , score AS number_of_upvotes
            , ROUND((score * (1 - upvote_ratio)) / upvote_ratio) AS number_of_downvotes
            , upvote_ratio
            , (1 - upvote_ratio) as downvote_ratio
            , num_comments AS number_of_comments
            , sentiment AS sentiment_category
            , sentiment_score
        FROM all_posts_with_sentiment
    ),

    all_posts_with_company_info AS (
        SELECT
            p.*
            , c.ticker_symbol
            , number_of_upvotes + number_of_downvotes AS total_number_of_votes
            , c.gics_sector
            , c.gics_sub_industry
        FROM all_posts_with_sentiment_transformed p
        INNER JOIN {catalog_name}.{schema_name}.{source_table_name[2]} c ON p.company_name = c.company_name
    )

    SELECT * FROM all_posts_with_company_info;
    """)

In [0]:
etl_logger.info(f'Loading: {silver_reddit_all_posts_df.count()} rows to: {target_table_name}')

(silver_reddit_all_posts_df.write.format("delta")
                           .mode("overwrite")
                           .partitionBy('created_date_utc')
                           .saveAsTable(f'{catalog_name}.{schema_name}.{target_table_name}'))